In [ ]:
import geopandas as gpd

# Load GeoJSON file
geojson_file = 'train.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'train.xlsx'
gdf.to_excel(xlsx, index=False)

# Load GeoJSON file
geojson_file = 'test.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'test.xlsx'
gdf.to_excel(xlsx, index=False)

In [ ]:
import pandas as pd
from shapely.geometry import Polygon

# Define the function to process a DataFrame and save to an Excel file
def processed_file(df, output_path):
    # Clean up the first two columns by removing spaces
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

    # Replace missing values with median for specified columns
    colonnes_a_traiter = df.columns[2:32]
    medianes = df[colonnes_a_traiter].median()
    df[colonnes_a_traiter] = df[colonnes_a_traiter].fillna(medianes)

    # Process the 'geometry' column to extract polygon coordinates
    def processv2(df):
        data = df['geometry']
        polygons = []

        for item in data:
            points = []
            item = item[10:-2]  # Remove unnecessary characters
            segments = item.split(',')

            for segment in segments:
                a, b = [coord for coord in segment.split(' ') if coord != '']
                points.append((float(a), float(b)))

            polygons.append(points)

        return polygons

    # Functions to calculate polygon metrics
    def calculate_polygon_area(vertices):
        polygon = Polygon(vertices)
        return polygon.area
    
    def calculate_polygon_perimeter(vertices):
        polygon = Polygon(vertices)
        return polygon.length
    
    def calculate_height_width_ratio(vertices):
        polygon = Polygon(vertices)
        bounds = polygon.bounds
        height = bounds[3] - bounds[1]
        width = bounds[2] - bounds[0]
        return height / width
    
    def calculate_centroid_x(vertices):
        polygon = Polygon(vertices)
        return polygon.centroid.x
    
    def calculate_centroid_y(vertices):
        polygon = Polygon(vertices)
        return polygon.centroid.y
    
    # Extract polygons from 'geometry' column
    polygons = processv2(df)
    
    # Add new columns to the DataFrame with polygon metrics
    df['Area'] = [calculate_polygon_area(polygon) for polygon in polygons]
    df['Perimeter'] = [calculate_polygon_perimeter(polygon) for polygon in polygons]
    df['ratio'] = [calculate_height_width_ratio(polygon) for polygon in polygons]
    df['x_centroid'] = [calculate_centroid_x(polygon) for polygon in polygons]
    df['y_centroid'] = [calculate_centroid_y(polygon) for polygon in polygons]
    
    # Drop 'geometry' column after extracting needed data
    df = df.drop(['geometry'], axis=1)

    # Update the first two columns with specified replacements
    df.iloc[:, 0] = df.iloc[:, 0].str.replace('N,A', 'NSP_urban')
    df.iloc[:, 1] = df.iloc[:, 1].str.replace('N,A', 'NSP_geo')

    # Apply one-hot encoding to 'urban_type' and 'geography_type'
    one_hot_encoded_urban = df['urban_type'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded_urban)

    one_hot_encoded_geo = df['geography_type'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded_geo)

    # Drop unneeded columns
    df = df.drop(columns=['urban_type', 'geography_type', 'index'])

    # Convert date columns to datetime objects
    df['date0'] = pd.to_datetime(df['date0'], format='%d-%m-%Y')
    df['date1'] = pd.to_datetime(df['date1'], format='%d-%m-%Y')
    df['date2'] = pd.to_datetime(df['date2'], format='%d-%m-%Y')
    df['date3'] = pd.to_datetime(df['date3'], format='%d-%m-%Y')
    df['date4'] = pd.to_datetime(df['date4'], format='%d-%m-%Y')

    # Calculate the day differences between date columns
    df['Ecart_jours1'] = (df['date1'] - df['date0']).dt.days
    df['Ecart_jours2'] = (df['date2'] - df['date1']).dt.days
    df['Ecart_jours3'] = (df['date3'] - df['date2']).dt.days
    df['Ecart_jours4'] = (df['date4'] - df['date3']).dt.days
    
    # Drop original date columns after calculating differences
    df = df.drop(columns=['date0', 'date1', 'date2', 'date3', 'date4'])

    # Apply one-hot encoding to 'change_status_dateX' columns
    one_hot_encoded0 = df['change_status_date0'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded0, rsuffix='0')
    
    one_hot_encoded1 = df['change_status_date1'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded1, rsuffix='1')
    
    one_hot_encoded2 = df['change_status_date2'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded2, rsuffix='2')

    one_hot_encoded3 = df['change_status_date3'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded3, rsuffix='3')

    one_hot_encoded4 = df['change_status_date4'].str.get_dummies(sep=',')
    df = df.join(one_hot_encoded4, rsuffix='4')
    
    # Drop the 'change_status_dateX' columns after one-hot encoding
    df = df.drop(columns=['change_status_date0', 'change_status_date1', 'change_status_date2', 'change_status_date3', 'change_status_date4'])
    
    # Colors for further calculations
    couleurs = ['red', 'blue', 'green']
    
    # Dates to consider for calculations
    dates = ['date1', 'date2', 'date3', 'date4', 'date5']
    
    # Loop to calculate mean for each color
    for couleur in couleurs:
        mean_values = []
        
        for index, row in df.iterrows():
            color_values = []
            
            for date in dates:
                column_name = f'img_{couleur}_mean_{date}'
                color_values.append(row[column_name])
            
            mean_values.append(sum(color_values) / len(color_values))
        
        df[f'moyenne_{couleur}'] = mean_values
    
    # Loop to calculate standard deviation for each color
    for couleur in couleurs:
        std_values = []
        
        for index, row in df.iterrows():
            color_values = []
            
            for date in dates:
                column_name = f'img_{couleur}_std_{date}'
                color_values.append(row[column_name])
            
            std_values.append(sum(color_values) / len(color_values))
        
        df[f'std_{couleur}'] = std_values
    
    # Save the processed DataFrame to an Excel file
    df.to_excel(output_path, index=False)  # Save without the index
